In [4]:
import pandas as pd
from difflib import SequenceMatcher
import re
import os
from nltk import sent_tokenize

# Load datasets
annotated_df = pd.read_excel(r"D:\66 CausalMap\Panasonic-IDS\data\MarketIntelligenceReport_Annotation_Tasks.xlsx", engine='openpyxl', sheet_name='MarketIntelligenceReport_Task1')
data = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR.csv")
df = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MarketIntelligenceReport.csv")
df['row_id'] = df.index
cols = ['Translated']
# duplicates, except first
m1 = df.duplicated(cols)
# duplicates including first
m2 = df.duplicated(cols, keep=False)
df['first_occurrence'] = (
 df[cols].merge(df.loc[~m1, cols+['row_id']],how='left')['row_id'].astype(str).where(m1, m2)
)

# Get conversions
rowid2docid = {v:k for k,v in enumerate(data['row_id'])}
def get_doc_id(row_id):
    if row_id not in rowid2docid:
        old = row_id
        row_id = int(df[df['row_id']==row_id]['first_occurrence'].values[0])
        print(f'convert {old}-->{row_id}')
    return rowid2docid[row_id]

In [5]:
# verify annotations row_id tallies with original
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
def find_ref_sent_and_sent_id(original, annotated, verbose=False):
    found = False
    for sent in annotated.split('\n'):
        if ('<CAUSE>' in sent) and ('<EFFECT' in sent):
            found = True
            break

    if not found:
        raise ValueError('No annotations found.')

    annotated_sent = sent
    sent = re.sub('<CAUSE>','',sent)
    sent = re.sub('</CAUSE>','',sent)
    sent = re.sub('<EFFECT>','',sent)
    sent = re.sub('</EFFECT>','',sent)
    
    # original before EDA and deduplication
    found = False
    for sent_id,ref_sent in enumerate(original.split('\n')):
        if similar(ref_sent, sent)>0.90:
            found = True
            break

    if not found:
        print(original)
        print(annotated_sent)
        raise ValueError('Similar original sentence not found.')

    # original after EDA and deduplication
    found = False
    reparsed_sent = sent_tokenize(re.sub('\.(?!\s|\d|$)','. ', sent))[0]
    reparsed_original = sent_tokenize(re.sub('\.(?!\s|\d|$)','. ', original))
    for reparsed_sent_id,reparsed_ref_sent in enumerate(reparsed_original):
        if similar(reparsed_ref_sent, reparsed_sent)>0.90:
            found = True
            break

    if not found:
        print(original)
        print(annotated_sent)
        raise ValueError('Similar original reparsed sentence not found.')
        
    if verbose:
        print(sent_id)
        print(ref_sent)
        print(annotated_sent)
    
    return sent_id, ref_sent, reparsed_sent_id,reparsed_ref_sent, annotated_sent


for i,row in annotated_df.iterrows():
    original = data.loc[get_doc_id(row.row_id)]['Text']
    annotated = row.annotations
    break

find_ref_sent_and_sent_id(original, annotated, verbose=False)

(1,
 'From to Uber and , many foreign companies have had to launch low-price versions of their products in a bid to make inroads in the Indian market.\r',
 1,
 'From to Uber and , many foreign companies have had to launch low-price versions of their products in a bid to make inroads in the Indian market.',
 '<EFFECT> From to Uber and , many foreign companies have had to launch low-price versions of their products </EFFECT> in a bid <CAUSE> to make inroads in the Indian market</CAUSE>.')

In [7]:
def remove_weird_chars(t):
    t = re.sub(r'"','',t)
    t = re.sub(r"'",'',t)
    t = re.sub(r"â|€|”|-|—|™|’|™|\x9d|œ",'',t)
    return t

In [8]:
def find_substring_in_text(text, substring):
    start = 0
    end = len(text)

    substring = re.sub(' ','',substring)
    substring = remove_weird_chars(substring)

    current_string = ''
    found = False
    for i,c in enumerate(text):
        
        c = remove_weird_chars(c)
        
        if c!=' ':
            current_string+=c
        else:
            continue
        
        n = len(current_string)
        if current_string==substring[:n]:
            if n==len(substring):
                found = True
                end = i+1 # end is inclusive
                break
            continue
        else:
            current_string=c
            start = i
    
    if not found:
        raise ValueError
    
    return start,end

In [9]:
# reformat annotations to ensure same as 'ref'/'original'
def reformat_annotations(text, annotated):
    
    cause = re.search(r'<CAUSE>(.*?)</CAUSE>', annotated).group(1).strip()
    effect = re.search(r'<EFFECT>(.*?)</EFFECT>', annotated).group(1).strip()
    
    cause_start, cause_end = find_substring_in_text(text, cause)
    effect_start, effect_end = find_substring_in_text(text, effect)

    adjustment = 13
    text_w_pairs = text
    if cause_start<effect_start:
        effect_start+=adjustment
        effect_end+=adjustment
        text_w_pairs = text_w_pairs[:cause_start] + '<ARG0>' + text_w_pairs[cause_start:cause_end] + '</ARG0>'+ text_w_pairs[cause_end:]
        text_w_pairs = text_w_pairs[:effect_start] + '<ARG1>' + text_w_pairs[effect_start:effect_end] + '</ARG1>'+ text_w_pairs[effect_end:]
    else:
        cause_start+=adjustment
        cause_end+=adjustment
        text_w_pairs = text_w_pairs[:effect_start] + '<ARG1>' + text_w_pairs[effect_start:effect_end] + '</ARG1>'+ text_w_pairs[effect_end:]
        text_w_pairs = text_w_pairs[:cause_start] + '<ARG0>' + text_w_pairs[cause_start:cause_end] + '</ARG0>'+ text_w_pairs[cause_end:]

#     text_w_pairs = text
#     text_w_pairs = re.sub(cause,f'<ARG0>{cause}</ARG0>',text_w_pairs)
#     text_w_pairs = re.sub(effect,f'<ARG1>{effect}</ARG1>',text_w_pairs)
    
    return text_w_pairs


sent_id, text, reparsed_sent_id, reparsed_text, annotated_sent = find_ref_sent_and_sent_id(original, annotated, verbose=False)
reformat_annotations(text, annotated_sent)

'<ARG1>From to Uber and , many foreign companies have had to launch low-price versions of their products</ARG1> in a bid <ARG0>to make inroads in the Indian market</ARG0>.\r'

In [10]:
from nltk.tokenize import sent_tokenize

In [11]:
cols = ['corpus','doc_id','sent_id','eg_id','index','text','text_w_pairs','seq_label','pair_label','context','num_sents']
ref = []
prev_id = None
eg_id = 0

for i,row in annotated_df.iterrows():
    
    
    doc_id = get_doc_id(row.row_id)
    original = data.loc[doc_id]['Text']
    
    annotated = row.annotations
    
    # sanity check
    if not similar(original, annotated)>0.89:
        raise ValueError(original,'\n',annotated)
    
    sent_id, text, reparsed_sent_id, reparsed_text, annotated_sent = find_ref_sent_and_sent_id(original, annotated, verbose=False)
    text_w_pairs = reformat_annotations(text, annotated_sent)
    
    curr_id = f'MIR_{doc_id}_{reparsed_sent_id}'
    if curr_id==prev_id:
        eg_id+=1
    else:
        eg_id=0
    prev_id = curr_id
    
    if ('<ARG0>' not in text_w_pairs) or ('<ARG1>' not in text_w_pairs):
        raise ValueError(f'Cause/Effect Arg not found: \n{text_w_pairs}')
    
    ref.append([
        'MIR',
        doc_id,
        reparsed_sent_id,
        eg_id,
        f'{curr_id}_{eg_id}',
        text,
        text_w_pairs,
        1,
        1,
        '',
        1
    ])

ref = pd.DataFrame(ref, columns=cols)
ref

convert 235-->145
convert 235-->145
convert 235-->145
convert 292-->95


,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
0,MIR,51,1,0,MIR_51_1_0,"From to Uber and , many foreign companies have...","<ARG1>From to Uber and , many foreign companie...",1,1,,1
1,MIR,51,6,0,MIR_51_6_0,"""Given these conditions, it is understandable ...","<ARG0>""Given these conditions</ARG0>, it is un...",1,1,,1
2,MIR,78,0,0,MIR_78_0_0,Toyota in India has largely pivoted toward hyb...,<ARG1>Toyota in India has largely pivoted towa...,1,1,,1
3,MIR,78,2,0,MIR_78_2_0,The government keeps taxes on cars and motorbi...,<ARG0>The government keeps taxes on cars and m...,1,1,,1
4,MIR,143,1,0,MIR_143_1_0,The Japanese automaker pioneered so-called Jus...,The Japanese automaker pioneered so-called Jus...,1,1,,1
5,MIR,143,3,0,MIR_143_3_0,From fashion to food processing to pharmaceuti...,<ARG0>From fashion to food processing to pharm...,1,1,,1
6,MIR,143,5,0,MIR_143_5_0,As the pandemic has hampered factory operation...,<ARG0>As the pandemic has hampered factory ope...,1,1,,1
7,MIR,94,6,0,MIR_94_6_0,"To kick-start the shift, the report suggests b...","To kick-start the shift, the report suggests <...",1,1,,1
8,MIR,328,0,0,MIR_328_0_0,Russian vehicle sales plunged by 63 percent in...,Russian vehicle sales plunged by 63 percent in...,1,1,,1
9,MIR,328,0,1,MIR_328_0_1,Russian vehicle sales plunged by 63 percent in...,<ARG1>Russian vehicle sales plunged by 63 perc...,1,1,,1


In [12]:
annotated_sent

'<EFFECT> ASEAN needs to take heed of past experiences </EFFECT>  in order <CAUSE>to be better prepared for future shocks </CAUSE>, he said, adding that one key lesson of the pandemic is the importance of a community approach.'

In [13]:
text

'ASEAN needs to take heed of past experiences in order to be better prepared for future shocks, he said, adding that one key lesson of the pandemic is the importance of a community approach.\r'

In [14]:
ref.to_csv(os.path.join(r'D:\66 CausalMap\Panasonic-IDS\data','MIR_annotated.csv'), index=False, encoding='utf-8-sig')

### Group Data

In [15]:
identifier_cols = ['corpus','doc_id','sent_id']

data2 = ref.copy()
grouped_causal_data = data2.pivot_table(
    index=identifier_cols+['text'], 
    columns='eg_id',
    aggfunc=lambda x: ' '.join(x), 
    values='text_w_pairs', 
    fill_value=0
).reset_index()

eg_id_cols = [c for c in grouped_causal_data.columns if c not in identifier_cols+['text']]
grouped_causal_data['causal_text_w_pairs'] = grouped_causal_data[eg_id_cols].values.tolist()
grouped_causal_data['eg_id'] = grouped_causal_data['causal_text_w_pairs'].apply(lambda x_list: min([i for i,x in enumerate(x_list) if x!=0]))
grouped_causal_data['causal_text_w_pairs'] = grouped_causal_data['causal_text_w_pairs'].apply(lambda x_list: [x for x in x_list if x!=0 and x!=""])
grouped_causal_data['num_rs'] = grouped_causal_data['causal_text_w_pairs'].apply(lambda x_list: int(len(x_list)))
grouped_causal_data = grouped_causal_data.drop(columns=eg_id_cols+['text'])
grouped_causal_data

eg_id,corpus,doc_id,sent_id,causal_text_w_pairs,eg_id,num_rs
0,MIR,51,1,"[<ARG1>From to Uber and , many foreign compani...",0,1
1,MIR,51,6,"[<ARG0>""Given these conditions</ARG0>, it is u...",0,1
2,MIR,78,0,[<ARG1>Toyota in India has largely pivoted tow...,0,1
3,MIR,78,2,[<ARG0>The government keeps taxes on cars and ...,0,1
4,MIR,94,6,"[To kick-start the shift, the report suggests ...",0,1
5,MIR,143,1,[The Japanese automaker pioneered so-called Ju...,0,1
6,MIR,143,3,[<ARG0>From fashion to food processing to phar...,0,1
7,MIR,143,5,[<ARG0>As the pandemic has hampered factory op...,0,1
8,MIR,328,0,[Russian vehicle sales plunged by 63 percent i...,0,2
9,MIR,387,2,[<ARG0>Wednesday’s vote will determine</ARG0> ...,0,1


In [16]:
grouped_ref = ref.merge(grouped_causal_data, how='right', on=identifier_cols+['eg_id'])
grouped_ref['num_rs'] = grouped_ref['num_rs'].fillna(0)
grouped_ref['num_rs'] = grouped_ref['num_rs'].astype(int)
grouped_ref

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents,causal_text_w_pairs,num_rs
0,MIR,51,1,0,MIR_51_1_0,"From to Uber and , many foreign companies have...","<ARG1>From to Uber and , many foreign companie...",1,1,,1,"[<ARG1>From to Uber and , many foreign compani...",1
1,MIR,51,6,0,MIR_51_6_0,"""Given these conditions, it is understandable ...","<ARG0>""Given these conditions</ARG0>, it is un...",1,1,,1,"[<ARG0>""Given these conditions</ARG0>, it is u...",1
2,MIR,78,0,0,MIR_78_0_0,Toyota in India has largely pivoted toward hyb...,<ARG1>Toyota in India has largely pivoted towa...,1,1,,1,[<ARG1>Toyota in India has largely pivoted tow...,1
3,MIR,78,2,0,MIR_78_2_0,The government keeps taxes on cars and motorbi...,<ARG0>The government keeps taxes on cars and m...,1,1,,1,[<ARG0>The government keeps taxes on cars and ...,1
4,MIR,94,6,0,MIR_94_6_0,"To kick-start the shift, the report suggests b...","To kick-start the shift, the report suggests <...",1,1,,1,"[To kick-start the shift, the report suggests ...",1
5,MIR,143,1,0,MIR_143_1_0,The Japanese automaker pioneered so-called Jus...,The Japanese automaker pioneered so-called Jus...,1,1,,1,[The Japanese automaker pioneered so-called Ju...,1
6,MIR,143,3,0,MIR_143_3_0,From fashion to food processing to pharmaceuti...,<ARG0>From fashion to food processing to pharm...,1,1,,1,[<ARG0>From fashion to food processing to phar...,1
7,MIR,143,5,0,MIR_143_5_0,As the pandemic has hampered factory operation...,<ARG0>As the pandemic has hampered factory ope...,1,1,,1,[<ARG0>As the pandemic has hampered factory op...,1
8,MIR,328,0,0,MIR_328_0_0,Russian vehicle sales plunged by 63 percent in...,Russian vehicle sales plunged by 63 percent in...,1,1,,1,[Russian vehicle sales plunged by 63 percent i...,2
9,MIR,387,2,0,MIR_387_2_0,Wednesday’s vote will determine how fast compa...,<ARG0>Wednesday’s vote will determine</ARG0> <...,1,1,,1,[<ARG0>Wednesday’s vote will determine</ARG0> ...,1


In [17]:
print('Sentences:', len(grouped_ref['num_rs']))
print('Relations:', sum(grouped_ref['num_rs']))

Sentences: 26
Relations: 28


In [18]:
grouped_ref.to_csv(os.path.join(r'D:\66 CausalMap\Panasonic-IDS\data','MIR_annotated_grouped.csv'), index=False, encoding='utf-8-sig')